In [293]:
import pandas as pd

In [294]:
result_df = pd.DataFrame(columns=['Prompt', 'source', 'target'])

In [295]:
forbidden_question_set_df = pd.read_csv('data/forbidden_question_set_df.csv')

In [296]:
forbidden_question_set_df.drop(['Unnamed: 0.1', 'Unnamed: 0', 'Length', 'Perplexity', 'embedding', 'idx'], axis=1, inplace=True)
forbidden_question_set_df['source'] = 'forbidden_questions'
forbidden_question_set_df['target'] = '1'
# forbidden_question_set_df['id'] = forbidden_question_set_df.index + 1

In [297]:
result_df = pd.concat([result_df, forbidden_question_set_df])

In [ ]:
result_df

In [299]:
jailbreak_prompts_df = pd.read_csv('data/jailbreak_prompts.csv')
jailbreak_prompts_df.drop(['Unnamed: 0', 'idx', 'Length', 'Perplexity', 'embedding'], axis=1, inplace=True)
jailbreak_prompts_df['source'] = 'jailbreak_prompts'
jailbreak_prompts_df['target'] = '1'

In [ ]:
result_df = pd.concat([result_df, jailbreak_prompts_df])
result_df

In [301]:
malicous_deepset_df = pd.read_csv('data/malicous_deepset.csv')
malicous_deepset_df.drop(['Unnamed: 0', 'idx', 'Length', 'Perplexity', 'embedding'], axis=1, inplace=True)

In [302]:
result_df = pd.concat([result_df, malicous_deepset_df])

In [303]:
predictionguard_df = pd.read_csv('data/predictionguard_df.csv')
predictionguard_df.drop(['Unnamed: 0', 'idx', 'Length', 'Perplexity', 'embedding'], axis=1, inplace=True)

In [ ]:
predictionguard_df

In [305]:
result_df = pd.concat([result_df, predictionguard_df])

In [306]:
given_train_promts = []
with open('data/student_solutions.txt') as f:
    given_train_promts = f.read().split('-----------------xyz-----------------')

In [307]:
# create a dataframe for given_train_promts
given_train_promts_df = pd.DataFrame(given_train_promts, columns=['Prompt'])
given_train_promts_df['source'] = 'row_given_train'
given_train_promts_df['target'] = '0'
result_df = pd.concat([result_df, given_train_promts_df])

# delete from memory all dataframes except result_df
del forbidden_question_set_df, jailbreak_prompts_df, malicous_deepset_df, predictionguard_df, given_train_promts_df

In [308]:
# leetcode_solutions_df = pd.read_csv('data/database_leetcode.csv')
# leetcode_solutions_df.drop(['Unnamed: 0', "Answer"], axis=1, inplace=True)
# leetcode_solutions_df.rename(columns={"Question": "Prompt"}, inplace=True)
# leetcode_solutions_df['source'] = 'leetcode'
# leetcode_solutions_df['target'] = '0'

In [309]:
# result_df = pd.concat([result_df, leetcode_solutions_df])
# del leetcode_solutions_df
# result_df

In [310]:
leetcode_pythons = pd.read_parquet('data/leetcode_solutions.parquet')['python_solutions'].to_list()

In [311]:
# create a dataframe for leetcode_pythons
leetcode_pythons_df = pd.DataFrame(leetcode_pythons, columns=['Prompt'])
leetcode_pythons_df['source'] = 'leetcode'
leetcode_pythons_df['target'] = '0'
result_df = pd.concat([result_df, leetcode_pythons_df])

del leetcode_pythons_df

In [312]:
smt_small_from_fug = pd.read_parquet('data/some_small_from_fug.parquete')
smt_small_from_fug.rename(columns={"text": "Prompt", "label":"target"}, inplace=True)
smt_small_from_fug['source'] ='smt_small_from_fug'

In [313]:
result_df = pd.concat([result_df, smt_small_from_fug])
del smt_small_from_fug
result_df.to_csv('data/result.csv', index=False)

In [ ]:
# check the median length of prompts for each source
result_df['prompt_length'] = result_df['Prompt'].str.len()
result_df.groupby('source')['prompt_length'].median()

In [315]:
import random

def merge_prompts_as_comments(initial_text, injection_text):
    # find all new lines in initial_text (separated by '\n')
    lines = initial_text.split('\n')
    # randomly choose a line to insert the injection text
    line_to_insert = random.choice(lines)
    # insert the injection text into the chosen line
    new_text = initial_text.replace(line_to_insert, line_to_insert + '\n' + "# " + injection_text)
    return new_text

def merge_prompts_as_containers(initial_text, injection_text):
    # find all new lines in initial_text (separated by '\n')
    lines = initial_text.split('\n')
    # randomly choose a line to insert the injection text
    line_to_insert = random.choice(lines)
    # insert the injection text into the chosen line
    new_text = initial_text.replace(line_to_insert, line_to_insert + '\n' + "string = " + '"'+injection_text+'"')
    return new_text

def merge_prompts_concatenate(initial_text, injection_text):
    # concatenate the two strings
    new_text = initial_text + injection_text
    return new_text
def merge_prompts_first(initial_text, injection_text):
    return injection_text + initial_text


In [319]:
infected_given_train = []
injections = result_df[result_df['source'] == 'jailbreak_prompts']['Prompt'].to_list()
for promt in given_train_promts:
    injections = random.sample(injections, min(len(injections), 10))
    infected_given_train.extend([merge_prompts_as_comments(promt, injection) for injection in injections])
    infected_given_train.extend([merge_prompts_concatenate(promt, injection) for injection in injections])
    infected_given_train.extend([merge_prompts_as_containers(promt, injection) for injection in injections])
    infected_given_train.extend([merge_prompts_first(promt, injection) for injection in injections])
    # cutted the lenght of injections
    infected_given_train.extend([merge_prompts_as_comments(promt, '.'.join(injection.split('.')[0:2])) for injection in injections])
    infected_given_train.extend([merge_prompts_concatenate(promt, '.'.join(injection.split('.')[0:2])) for injection in injections])
    infected_given_train.extend([merge_prompts_as_containers(promt, '.'.join(injection.split('.')[0:2])) for injection in injections])
    infected_given_train.extend([merge_prompts_first(promt, '.'.join(injection.split('.')[0:2])) for injection in injections])


In [324]:
# add the infected prompts to the result dataframe
infected_given_train_df = pd.DataFrame(infected_given_train, columns=['Prompt'])
infected_given_train_df['source'] = 'infected_given_train'
infected_given_train_df['target'] = '1'
result_df = pd.concat([result_df, infected_given_train_df])

# save the result dataframe
result_df.to_csv('data/result.csv', index=False)

In [ ]:
# count the number of rows in each source category
result_df['source'].value_counts()

In [ ]:

short_df = pd.concat([
    result_df[result_df['source'] == 'forbidden_questions'].sample(n=1000),
    result_df[result_df['source'] == 'leetcode'].sample(n=2000),
    result_df[result_df['source'] == 'infected_given_train'].sample(n=3000),
    result_df[result_df['source'] == 'row_given_train'].sample(n=347),
    result_df[result_df['source'] == 'jailbreak_prompts'].sample(n=2000)
])
short_df.to_csv('data/short_data.csv', index=False)